In [1]:
import sys
from importlib import reload

try:
    reload(sys.modules['utils']);
    reload(sys.modules['mds.cmds']);
    reload(sys.modules['mds.lmds']);
    reload(sys.modules['trainHelper']);
    reload(sys.modules['lossFunction']);
except:
    pass

import trainHelper
import utils
import numpy
from torch.utils.data import DataLoader
import torch
from torch import nn, Tensor
import lossFunction as lossF
from model.AutoEncoder import AutoEncoder
from model.DynParam import DynParam
from model.Linear import Linear
from model.VAE import VAE
import os.path 
import glob


In [3]:
sample_space = (1000, 1)
ss, N, d = 3200, 10, 2

In [4]:
try:
    euclidean_data1 = utils.load_variable('data/euclidean_data1.pkl')
    euclidean_data2 = utils.load_variable('data/euclidean_data2.pkl')
    
    rand_data1 = utils.load_variable('data/rand_data.pkl')
    rand_data2 = utils.load_variable('data/rand_data.pkl')
    
    if euclidean_data.size() != (ss, 1, N, N):
        print("Updated data for requirement !")
        raise Exception("Previous data not match requirement !")

except:
    euclidean_data1 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    euclidean_data2 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data1 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data2 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    utils.dump_variable(euclidean_data1, 'data/euclidean_data1.pkl')
    utils.dump_variable(euclidean_data2, 'data/euclidean_data2.pkl')

    utils.dump_variable(rand_data1, 'data/rand_data1.pkl')
    utils.dump_variable(rand_data2, 'data/rand_data2.pkl')

In [ ]:
data = torch.stack([
            # euclidean_data1,
            # euclidean_data2,
            rand_data1,
            rand_data2,
        ]) 

data = data.view(ss * 2, 1, N, N)

data = utils.minmax_norm(data, dmin=0)[0]
data = torch.tensor(data.data, requires_grad=True)

batch = 32
dlr = DataLoader(data, batch_size=batch, shuffle=True)

data[0, 0, :3, :3].data

In [1]:
class CoordsToDMLoss(nn.Module):
    
    def __init__(self, N, d):
        super(CoordsToDMLoss, self).__init__()
        
        self.N = N
        self.d = d

    def forward(self, rs, target_dm):

        batch = target_dm.size()[0]

        rs = rs.view(batch, self.N, self.d)

        model_dm = utils.get_distance_matrix(rs)
        model_dm = utils.minmax_norm(model_dm, dmin=0)[0]

        target_dm = torch.pow(target_dm, 2)    
        target_dm = utils.minmax_norm(target_dm, dmin=0)[0]

        loss = model_dm - target_dm.view_as(model_dm)
        loss = torch.pow(loss, 2)

        loss = loss.view(loss.size()[0], 1, -1)
        return torch.sum(torch.sum(loss, dim=2) ** 2)


NameError: name 'nn' is not defined

In [ ]:
class ReconLoss(nn.Module):
    
    def __init__(self):

        super(ReconLoss, self).__init__()

    def forward(self, rs, target_dm):
        
        model_dm = rs
        model_dm = utils.minmax_norm(model_dm, dmin=0)[0]
        
        target_dm = torch.pow(target_dm, 2)    
        target_dm = utils.minmax_norm(target_dm, dmin=0)[0]
        
        target_dm = utils.vectorize_distance_from_DM(target_dm)

        loss = model_dm - target_dm.view_as(model_dm)
        loss = torch.pow(loss, 2)

        loss = loss.view(loss.size()[0], 1, -1)
        return torch.sum(torch.sum(loss, dim=2) ** 2)


In [ ]:

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "Linear_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) / 2) 
        out_dim = N * 2

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU,
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = CoordsToDMLoss(N, 2)
        
        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())

In [ ]:

def preprocess(x):
    return torch.tensor(x.data, requires_grad=True)
    
for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "Linear_" + str(i)+ "_" + str(neuron) + "_matrix_LReLU"

        in_dim = N
        out_dim = 2

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = CoordsToDMLoss(N, 2)

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break
                
            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())   

In [ ]:
def preprocess(x):
    batch = x.size()[0]
    N = x.size()[-1]

    ex = utils.eignmatrix(x.view(batch, N, N)).view(batch, 1, N, N * 2)
    return torch.tensor(ex.data, requires_grad=True)
    
for neuron in [32]:

    for i in range(6, 12):

        model_id = "Linear_" + str(i)+ "_" + str(neuron) + "_eign_LReLU"

        in_dim = N * 2
        out_dim = 2

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = CoordsToDMLoss(N, 2)

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break
                
            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())   

In [6]:
data = torch.stack([
            euclidean_data1,
            euclidean_data2,
            # rand_data1,
            # rand_data2,
        ])
        
data = data.view(ss * 2, 1, N, N)

data = utils.minmax_norm(data, dmin=0)[0]
data = torch.tensor(data.data, requires_grad=True)

batch = 32
dlr = DataLoader(data, batch_size=batch, shuffle=True)

In [ ]:

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "Linear_AE_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) /2) 
        out_dim = int((N * N - N) /2) 

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = ReconLoss()

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break
                
            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())

In [ ]:

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "AE_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = AutoEncoder([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = ReconLoss()

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())

In [ ]:
raise Exception("End")

def MSES(model_dm, target_dm):
    
    model_dm = utils.minmax_norm(model_dm)[0]
    return nn.functional.binary_cross_entropy(model_dm, target_dm, reduction='sum')

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "VAE_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = VAE([in_dim, *[neuron for j in range(i)], out_dim],
                    activation=nn.LeakyReLU,
                    final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        
        lossFun = lossF.VAELoss(lossFun=MSES)

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(3):

            EPOCH = 300
            
            print("Training ", helper.id)

            helper.train(dlr, EPOCH, print_on_each=100)
            helper.backup()

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s")    


In [10]:
l = torch.triu(data[:3])

In [12]:
l

tensor([[[[0.0000, 0.3328, 0.2007, 0.5859, 0.1062, 0.6188, 0.3526, 0.6922,
           0.4985, 0.1804],
          [0.0000, 0.0000, 0.4270, 0.8992, 0.4089, 0.9422, 0.6372, 1.0000,
           0.8310, 0.3131],
          [0.0000, 0.0000, 0.0000, 0.4882, 0.1153, 0.5453, 0.4846, 0.5818,
           0.4803, 0.1183],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.4923, 0.0868, 0.5583, 0.1100,
           0.2175, 0.6056],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5333, 0.3745, 0.5960,
           0.4330, 0.1604],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5388, 0.1373,
           0.1782, 0.6601],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6629,
           0.3623, 0.5221],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.3099, 0.7000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.5822],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000